In [89]:

# Utility Imports
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
# Supervised Learning 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
# Unsupervised Learning
from sklearn.cluster import KMeans
import pprint

with open('matches_all.json') as f:
    data = json.load(f)
    matches = data["matches"]
    
    # analye pre-game + post-game information
    # utilize random forests for categorical data
    # n_estimators = 1,2,5,10,20,50 (find optimal number of trees)
    
    # order of postgame features: (see report)
    match_totals_data = []
    match_totals_target = []
    
    # put with respect to teamId == 100
    for match_idx in range(len(matches)):
        if matches[match_idx]["teams"][0]["win"] == "Win":
            match_totals_target += [1]
        else:
            match_totals_target += [0]
        
        blue_creep_diff = 0
        # if blue side had more damage dealt to champs or not (or dmg dealt differential)
        blue_total_dmg = 0
        red_total_dmg = 0
        blue_total_gold = 0
        red_total_gold = 0
        blue_avg_champ_level = 0
        red_avg_champ_level = 0
        blue_total_kills = 0
        red_total_kills = 0
        blue_total_deaths = 0
        red_total_deaths = 0
        blue_total_assists = 0
        red_total_assists = 0
        for player in range(len(matches[match_idx]["participants"])):
            if(player < 5):
                if(matches[match_idx]["participants"][player]["timeline"]["lane"] == "BOTTOM" and matches[match_idx]["participants"][player]["timeline"]["role"] == "DUO_SUPPORT"):
                    blue_total_dmg += matches[match_idx]["participants"][player]["stats"]["totalDamageDealtToChampions"]
                    blue_total_gold += matches[match_idx]["participants"][player]["stats"]["goldEarned"]
                    blue_avg_champ_level += matches[match_idx]["participants"][player]["stats"]["champLevel"]
                    blue_total_kills += matches[match_idx]["participants"][player]["stats"]["kills"]
                    if(matches[match_idx]["participants"][player]["stats"]["deaths"] == 0):
                        blue_total_deaths += 1
                    else:
                        blue_total_deaths += matches[match_idx]["participants"][player]["stats"]["deaths"]
                    blue_total_assists += matches[match_idx]["participants"][player]["stats"]["assists"]
            else:
                if(matches[match_idx]["participants"][player]["timeline"]["lane"] == "BOTTOM" and matches[match_idx]["participants"][player]["timeline"]["role"] == "DUO_SUPPORT"):
                    red_total_dmg += matches[match_idx]["participants"][player]["stats"]["totalDamageDealtToChampions"]
                    red_total_gold += matches[match_idx]["participants"][player]["stats"]["goldEarned"]
                    red_avg_champ_level += matches[match_idx]["participants"][player]["stats"]["champLevel"]
                    red_total_kills += matches[match_idx]["participants"][player]["stats"]["kills"]
                    if(matches[match_idx]["participants"][player]["stats"]["deaths"] == 0):
                        red_total_deaths += 1
                    else:
                        red_total_deaths += matches[match_idx]["participants"][player]["stats"]["deaths"]
                    red_total_assists += matches[match_idx]["participants"][player]["stats"]["assists"]

        
    
        # uncomment lines as we are trying to consider features
        match_totals_data += [[blue_total_dmg - red_total_dmg]]
#         match_totals_data += [[blue_total_gold - red_total_gold]]
        match_totals_data[match_idx] += [blue_total_gold - red_total_gold]
#         match_totals_data += [[(blue_avg_champ_level/5)-(red_avg_champ_level/5)]]
        match_totals_data[match_idx] += [(blue_avg_champ_level/5)-(red_avg_champ_level/5)]
#         if(blue_total_deaths == 0 or red_total_deaths == 0):
#             match_totals_data += [[0]]
#         else:
#             match_totals_data += [[((blue_total_kills+blue_total_assists)/blue_total_deaths)-(red_total_kills+red_total_assists)/red_total_deaths]]
#         if(blue_total_deaths == 0 or red_total_deaths == 0):
#             match_totals_data[match_idx] += [0]
#         else:
#             match_totals_data[match_idx] += [((blue_total_kills+blue_total_assists)/blue_total_deaths)-(red_total_kills+red_total_assists)/red_total_deaths]
#         match_totals_data[match_idx] += [((blue_total_kills+blue_total_assists)/blue_total_deaths)-(red_total_kills+red_total_assists)/red_total_deaths]

        # if blue side had more gold at the end of the game or not (or team gold differential)
    
    ########## OVERTIME ################
    match_overtime_data = []
    match_overtime_target = []
    matches_indices = []
    avg_cs_diff = 0
    avg_dmg_taken = 0
    avg_exp_diff = 0
    
    for match_idx in range(len(matches)):
        if(matches[match_idx]["teams"][0]["win"] == "Win"):
            temp = []
            for player in range(5):
                if('csDiffPerMinDeltas' in matches[match_idx]["participants"][player]["timeline"] and 'damageTakenDiffPerMinDeltas' in matches[match_idx]["participants"][player]["timeline"] and 'xpDiffPerMinDeltas' in matches[match_idx]["participants"][player]["timeline"]):
                    temp.append(True)
                else:
                    temp.append(False)
            if(len(temp) == temp.count(temp[0]) and temp[0] == True):
                match_overtime_target += [1]
                matches_indices += [match_idx]
        else:
            temp = []
            for player in range(5):
                if('csDiffPerMinDeltas' in matches[match_idx]["participants"][player]["timeline"] and 'damageTakenDiffPerMinDeltas' in matches[match_idx]["participants"][player]["timeline"] and 'xpDiffPerMinDeltas' in matches[match_idx]["participants"][player]["timeline"]):
                    temp.append(True)
                else:
                    temp.append(False)
            if(len(temp) == temp.count(temp[0]) and temp[0] == True):
                match_overtime_target += [0]
                matches_indices += [match_idx]
    
    iteration = 0
    for match in matches_indices:
        counter = 0
        cs_total_diff = 0
        dmg_taken_diff = 0
        exp_diff = 0
        for player in range(5):
            for i in matches[match]["participants"][player]["timeline"]['csDiffPerMinDeltas']:
                counter += 1
                cs_total_diff += matches[match]["participants"][player]["timeline"]['csDiffPerMinDeltas'][i]
                dmg_taken_diff += matches[match]["participants"][player]["timeline"]['damageTakenDiffPerMinDeltas'][i]
                exp_diff += matches[match]["participants"][player]["timeline"]['xpDiffPerMinDeltas'][i]
        
        # uncomment lines as we are trying to consider features
#         match_overtime_data += [[cs_total_diff/counter]]
#         match_overtime_data[iteration] += [dmg_taken_diff/counter]
#         match_overtime_data += [[dmg_taken_diff/counter]]
#         match_overtime_data[iteration] += [exp_diff/counter]
        match_overtime_data += [[exp_diff/counter]]
        iteration += 1
        avg_cs_diff += cs_total_diff/counter
        avg_dmg_taken += dmg_taken_diff/counter
        avg_exp_diff += exp_diff/counter
    
    avg_cs_diff = avg_cs_diff/len(matches_indices)
    avg_dmg_taken = avg_dmg_taken/len(matches_indices)
    avg_exp_diff = avg_exp_diff/len(matches_indices)
    
#     to_add = [avg_cs_diff,avg_dmg_taken,avg_exp_diff]
    to_add = [avg_exp_diff]
    not_in = [range(len(matches))[i] for i in range(len(matches)) if range(len(matches))[i] not in matches_indices]
    to_add_targets = [match_totals_target[i] for i in not_in]
    
    for i in range(len(not_in)):
        match_overtime_data.append(to_add)
    match_overtime_target += to_add_targets
    
    print(len(match_overtime_data))
    print(len(match_overtime_target))
    
    # collect and scale data
#     X_train, X_test, Y_train, Y_test = train_test_split(match_ingame_data,match_target,random_state=0)
    X_train, X_test, Y_train, Y_test = train_test_split(match_totals_data,match_totals_target,random_state=0)
#     X_train, X_test, Y_train, Y_test = train_test_split(match_overtime_data,match_overtime_target,random_state=0)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_transformed = scaler.transform(X_train)
    X_test_transformed = scaler.transform(X_test)
    
    forestModel = RandomForestClassifier(n_estimators=50,max_features=3,random_state=0).fit(X_train_transformed,Y_train)
    # Log model potential improvement: regularization parameter
    best_score = 0
    best_alpha = 0
    kfolds = 5
    for param in [0.01,0.1,1,10,100]:
        logModel = LogisticRegression(C=param)
        scores = cross_val_score(logModel,X_train_transformed,Y_train,cv=kfolds)
        score = np.mean(scores)
        if(score > best_score):
            best_score = score
            best_alpha = param
    SelectedLogModel = LogisticRegression(C=best_alpha).fit(X_train_transformed,Y_train)
    print("accuracy = ", forestModel.score(X_test_transformed,Y_test))
    print("best alpha = ", best_alpha)
    # pprint.pprint(matches[199]["participants"][0]) <- sample usage of JSON file dictionaries

1000
1000
accuracy =  0.852
best alpha =  0.01
True
{'championId': 79,
 'highestAchievedSeasonTier': 'SILVER',
 'masteries': [{'masteryId': 6114, 'rank': 5},
               {'masteryId': 6121, 'rank': 1},
               {'masteryId': 6211, 'rank': 5},
               {'masteryId': 6223, 'rank': 1},
               {'masteryId': 6231, 'rank': 5},
               {'masteryId': 6241, 'rank': 1},
               {'masteryId': 6251, 'rank': 5},
               {'masteryId': 6262, 'rank': 1},
               {'masteryId': 6312, 'rank': 5},
               {'masteryId': 6321, 'rank': 1}],
 'participantId': 1,
 'runes': [{'rank': 9, 'runeId': 5273},
           {'rank': 9, 'runeId': 5289},
           {'rank': 9, 'runeId': 5317},
           {'rank': 3, 'runeId': 5357}],
 'spell1Id': 11,
 'spell2Id': 4,
 'stats': {'assists': 6,
           'champLevel': 11,
           'combatPlayerScore': 0,
           'damageDealtToObjectives': 3484,
           'damageDealtToTurrets': 739,
           'damageSelfMitigate